In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

from time import sleep, time
from threading import Thread
from scipy.cluster.hierarchy import fcluster
from scipy.stats import zscore

from data_processing import DataIterator
from testing import TestFactory, ClusteredInfo
from dtw import dtw as cur_dtw
from dtw_wrapper import DtwWrapper

In [2]:
def norm_1(x, y):
    return np.linalg.norm(x - y, ord=1)
def norm_2(x, y):
    return np.linalg.norm(x - y, ord=2)

# Rewriting data in our format

In [3]:
# data = pd.read_csv("../data/preprocessed_large.csv", header=None)

# indexes = pd.Index(range(0, 200))
# data_preprocessed = pd.DataFrame(columns=["obj", "ch", "label", *indexes])

# l = []
# for j in data.index:
#     df = pd.DataFrame({"obj": [j for i in range(3)], 
#                   "label": [data.loc[j, 0] for i in range(3)]},
#                  columns=["obj", "label", *indexes])
#     df.index.name = "ch"
#     df.loc[0, indexes] = data.loc[j, 1:200].values
#     df.loc[1, indexes] = data.loc[j, 201:400].values
#     df.loc[2, indexes] = data.loc[j, 401:600].values 
#     df.reset_index(inplace=True)
#     data_preprocessed = data_preprocessed.append(df, sort=False)
    
# data_preprocessed.reset_index(drop=True, inplace=True)


In [4]:
data = pd.read_csv("../data/preprocessed_3ch_50point_overlap0.csv", index_col=0)
it = DataIterator(data, 50, random_state=42)
tests = TestFactory(it=it)
x = tests.set_sample(2048)

In [5]:
wrap = DtwWrapper(tests.X, hash(tests.infos), cur_dtw, norm_1, dtw_args={"z_normalize": True, "l": 0.2})
wrap.fill_distances(n_threads=8)

wrap = DtwWrapper(tests.X, hash(tests.infos), cur_dtw, norm_2, dtw_args={"z_normalize": True, "l": 0.2})
wrap.fill_distances(n_threads=8)

dump
0
dump
30
dump
60
dump
90
dump
120
dump
150
dump
180
dump
210
dump
240
dump
270
dump
300
dump
330
dump
360
dump
390
dump
420
dump
450
dump
480
dump
510
dump
540
dump
570
dump
600
dump
630
dump
660
dump
690
dump
720
dump
750
dump
780
dump
810
dump
840
dump
870
dump
900
dump
930
dump
960
dump
990
42530.911413669586
dump
0
dump
30
dump
60
dump
90
dump
120
dump
150
dump
180
dump
210
dump
240
dump
270
dump
300
dump
330
dump
360
dump
390
dump
420
dump
450
dump
480
dump
510
dump
540
dump
570
dump
600
dump
630
dump
660
dump
690
dump
720
dump
750
dump
780
dump
810
dump
840
dump
870
dump
900
dump
930
dump
960
dump
990
51482.45946884155


In [7]:
tmp = np.genfromtxt("../data/distances/dtwnorm_1z_normalizeTruel0.2-43544539162478229291.csv")
(tmp < 0).sum() / (tmp != -1000).sum()

0.0

In [8]:
info = tests.test_dtw(cur_dtw, norm_2, 
                      dtw_args={"z_normalize": True, "l": 0.2}, cluster_dist="weighted")

Loaded
Loaded
Elapsed time: 4.790


In [9]:
info.cluster(50)
info.stats
index = info.stats.head(10).index.values
classifier_stat = {}
for i in info.stats.index:
    classifier_stat[i] = pd.Series(info.label[np.where(info.clusters_labels == i)[0]]).value_counts()
    
pd.DataFrame(classifier_stat).fillna(0).iloc[:, :30]

,43,45,30,37,44,50,35,16,27,21,...,4,48,17,20,10,18,3,13,2,23
1,2,7,2,93,1.0,43,0.0,68.0,55.0,1.0,...,0.0,1.0,1,0.0,0.0,8.0,0.0,0.0,0.0,0.0
2,42,99,157,7,7.0,31,1.0,3.0,0.0,0.0,...,0.0,1.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,39,44,19,44,109.0,8,14.0,11.0,5.0,1.0,...,0.0,2.0,8,2.0,1.0,0.0,0.0,0.0,2.0,0.0
4,152,72,1,32,15.0,18,19.0,6.0,2.0,6.0,...,0.0,0.0,3,6.0,0.0,0.0,7.0,0.0,0.0,3.0
5,2,7,13,3,3.0,12,9.0,2.0,6.0,29.0,...,6.0,1.0,6,3.0,11.0,6.0,4.0,5.0,4.0,3.0
6,2,9,2,8,0.0,12,49.0,0.0,0.0,24.0,...,17.0,16.0,1,6.0,5.0,3.0,5.0,11.0,8.0,5.0


In [13]:
info = tests.test_dtw(cur_dtw, norm_1, 
                      dtw_args={"z_normalize": True, "l": 0.2},)

Loaded
Elapsed time: 4.715


In [14]:
info.cluster(24)
info.stats
index = info.stats.head(10).index.values
classifier_stat = {}
for i in info.stats.index:
    classifier_stat[i] = pd.Series(info.label[np.where(info.clusters_labels == i)[0]]).value_counts()
    
pd.DataFrame(classifier_stat).fillna(0).iloc[:, :30]

,12,19,21,14,11,22,20,9,13,23,...,5,2,8,15,3,1,4,16,17,10
1,16,55,40,116,3,98.0,5.0,3.0,0.0,2,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,121,45,27,54,94,0.0,0.0,0.0,0.0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,80,28,45,34,44,15.0,21.0,17.0,7.0,3,...,1.0,0.0,2.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
4,77,63,88,9,42,20.0,17.0,8.0,3.0,5,...,7.0,0.0,6.0,1.0,0.0,4.0,2.0,0.0,1.0,0.0
5,13,40,15,4,9,20.0,26.0,26.0,27.0,16,...,7.0,15.0,11.0,18.0,4.0,6.0,9.0,4.0,7.0,7.0
6,23,27,23,13,9,11.0,17.0,14.0,21.0,21,...,15.0,14.0,7.0,7.0,21.0,9.0,7.0,12.0,6.0,3.0
